This code is for analyzing MEA neurotoxicity screening data and generating an objective index - neural activity score - to evaluate network ontogeny and effects of treatments/conditions/perturbation.

See Passaro et al. and the README file in the GitHub repository for more information, usage instructions, and sample data.

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import os
import shutil
import openpyxl as xl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import csv
import numpy as np
from scipy import stats
import sys
from collections import OrderedDict

In [ ]:
%%capture
from tqdm import tqdm
tqdm().pandas()

In [ ]:
# Ask which directory to read from, note the following file structure below:
# Experiment folder > Plate subfolder(s) > DIV subfolder(s) > Maestro.raw, StatCompiler.csv, etc...
# Select the EXPERIMENT folder
print('Select experiment folder...')
root = tk.Tk()
root.withdraw()
experimentPath = filedialog.askdirectory()
print("Experiment folder: ",experimentPath)

In [ ]:
### Compile .csv StatCompiler files into one .xlsx file ###
def compileAndMerge(experimentPath):
    # List compound subfolders
    compoundFolders = os.listdir(experimentPath)
    compoundFolders = [x for x in compoundFolders if '.datastreams' not in x]
    compoundFolders = [x for x in compoundFolders if '.xlsx' not in x]
    compoundFolders = [x for x in compoundFolders if 'Compiled' not in x]
    compoundPaths = [experimentPath + '\\' + x for x in compoundFolders]

    for path in tqdm(compoundPaths):
        # List plate subfolders
        plateFolders = os.listdir(path)
        plateFolders = [x for x in plateFolders if '.csv' not in x]
        plateFolders = [x for x in plateFolders if '.raw' not in x]
        plateFolders = [x for x in plateFolders if '.txt' not in x]
        plateFolders = [x for x in plateFolders if '.xlsx' not in x]
        plateFolders = [x for x in plateFolders if 'Compiled' not in x]
        
        # Loop through each plate folder
        for plateFolder in plateFolders:
            if 'platemap' in plateFolder:	# Ignore platemap file(s)
                continue
            else:
                platePath = path + '\\' + plateFolder
            
            # Create Compiled plate .xlsx file
            fileout = platePath + '\\' + '1_AllDaysCompiled_' + plateFolder + '.xlsx'
            writer = pd.ExcelWriter(fileout, engine='xlsxwriter')

            # Create list of day (DIV) folders within plate folder
            divFolders = os.listdir(platePath)
            divFolders = [x for x in divFolders if '.csv' not in x]
            divFolders = [x for x in divFolders if '.xlsx' not in x]

            # Go into each DIV folder and list filenames
            for div in divFolders:
                files = os.listdir(platePath + '\\' + div)

            # Find .csv files
                for filename in files:
                    if '.csv' in filename and 'Statistics' in filename:	# and '(000)' in filename:
                        # Copy file to destination folder
                        sourceFile = platePath + '\\' + div + '\\' + filename
                        destinationFile = platePath + '\\' + div + '_' + filename
                        shutil.copy(sourceFile, destinationFile)

                        # Merge file into Compiled .xlsx file
                        df = pd.read_csv(sourceFile, names = list(range(0,770)), header=None, engine='python')
                        df.to_excel(writer, sheet_name=div, header=False, index=False)
            writer.save()
    print('All Excel files compiled.')

In [ ]:
compileAndMerge(experimentPath)

In [ ]:
### Analyze compiled .xslx files for all parameters ###
def analyzeStatCompilerFiles(experimentPath):
    # List compound subfolders
    compoundFolders = os.listdir(experimentPath)
    compoundFolders = [x for x in compoundFolders if '.datastreams' not in x]
    compoundFolders = [x for x in compoundFolders if '.xlsx' not in x]
    compoundPaths = [experimentPath + '\\' + x for x in compoundFolders]

    for path in tqdm(compoundPaths):    
        # List plate subfolders
        plateFolders = os.listdir(path)
        plateFolders = [x for x in plateFolders if '.csv' not in x]
        plateFolders = [x for x in plateFolders if '.raw' not in x]
        plateFolders = [x for x in plateFolders if '.txt' not in x]
        plateFolders = [x for x in plateFolders if '.xlsx' not in x]
        plateFolders = [x for x in plateFolders if 'Compiled' not in x]

        # Loop through each plate folder
        for plateFolder in plateFolders:
            if 'platemap' in plateFolder:	# Ignore platemap file(s)
                continue
            else:
                platePath = path + '\\' + plateFolder

            filename = platePath + '\\' + '1_AllDaysCompiled_' + plateFolder + '.xlsx'
            wb = xl.load_workbook(filename) 	# Load first workbook (one plate compiled, each sheet = 1 day)
            sheets = wb.sheetnames

            ##### Determine array dimensions (reps, groups, parameters, days) #####
            # Read groups/treatments from ExperimentLog files
            sheet = wb.worksheets[0]
            groups = []                 # Define empty list to store group names
            for row in range(1, sheet.max_row+1): # Loop through rows to find row containing Group/Treatment names
                if sheet.cell(row = row, column = 1).value == 'Treatment/ID':
                    groupRow = row
                    break
            for row in range(1, sheet.max_row+1): # Loop through rows to find row containing Group/Treatment names
                if sheet.cell(row = row, column = 1).value == 'Treatment':
                    compoundRow = row
                    break  
            for row in range(1, sheet.max_row+1): # Loop through rows to find row containing Concentration values
                if sheet.cell(row = row, column = 1).value == 'Concentration':
                    concentrationRow = row
                    break
                    
            for file in os.listdir(path):
                if plateFolder in file and 'MaestroExperimentLog' in file and '.csv' in file and '.xlsx' not in file:
                    wbNew = Workbook()
                    wsNew = wbNew.active
                    with open(path + '\\' + file, 'r') as f:
                        for row in csv.reader(f):
                            wsNew.append(row)
                    wbNew.save(path + '\\' + file + '.xlsx')
            for file in os.listdir(path):
                if plateFolder in file and 'MaestroExperimentLog' in file and '.xlsx' in file:
                    wbExpLog = xl.load_workbook(path + '\\' + file)
                    expLogSheet = wbExpLog.worksheets[0]
                    break

            for column in range(1,expLogSheet.max_column+1):
                if expLogSheet.cell(row = 1, column = column).value == 'Treatment':
                    treatmentCol = column
                    break
            for column in range(2,expLogSheet.max_column+1):        
                if expLogSheet.cell(row = 1, column = column).value == 'Dose' or expLogSheet.cell(row = 1, column = column).value == 'Dose ':
                    doseCol = column
                    break
            
            for row in range(2,expLogSheet.max_row+1):
                if expLogSheet.cell(row = row, column = doseCol).value == '3' or expLogSheet.cell(row = row, column = doseCol).value == 3:
                    expLogSheet.cell(row = row, column = doseCol).value = '03'
                if expLogSheet.cell(row = row, column = doseCol).value == '1' or expLogSheet.cell(row = row, column = doseCol).value == 1:
                    expLogSheet.cell(row = row, column = doseCol).value = '01'
                    
            for column in range(2,expLogSheet.max_column+1):
                if expLogSheet.cell(row = 1, column = column).value == 'Units':
                    unitsCol = column
                    break

            for column in range(2,sheet.max_column+1):
                sheet.cell(row = groupRow, column = column).value = str(expLogSheet.cell(row = column, column = treatmentCol).value) + ' - ' + str(expLogSheet.cell(row = column, column = doseCol).value) + ' ' + str(expLogSheet.cell(row = column, column = unitsCol).value)
                sheet.cell(row = compoundRow, column = column).value = str(expLogSheet.cell(row = column, column = treatmentCol).value)
                sheet.cell(row = concentrationRow, column = column).value = expLogSheet.cell(row = column, column = doseCol).value
                
            for column in range(2,sheet.max_column+1):
                if sheet.cell(row = groupRow, column = column).value != None and 'None' not in str(sheet.cell(row = groupRow, column = column).value) and sheet.cell(row = groupRow, column = column).value not in groups:
                    groups.append(str(sheet.cell(row = groupRow, column = column).value))

            # Determine number of groups (if not defined in first sheet, try to copy from last sheet first)
            numgroups = len(groups)
            if numgroups == 0:
                lastSheet = wb.worksheets[-1]
                for column in range(2,lastSheet.max_column+1):
                    if lastSheet.cell(row = groupRow, column = column).value != None:
                        sheet.cell(row = groupRow, column = column).value = lastSheet.cell(row = groupRow, column = column).value
            for column in range(2,sheet.max_column+1):
                if sheet.cell(row = groupRow, column = column).value != None and 'None' not in str(sheet.cell(row = groupRow, column = column).value) and sheet.cell(row = groupRow, column = column).value not in groups:
                    groups.append(str(sheet.cell(row = groupRow, column = column).value))

            numgroups = len(groups)
            if numgroups == 0:
                raise Warning("Define groups in first sheet of .xlsx file ('Treatment' row) before proceeding.")
                sys.exit()

            # Find number of reps (assumes equal number per group)
            for row in range(1, sheet.max_row+1):
                if sheet.cell(row = row, column = 1).value == 'Total Wells':
                    numWells = 0
                    if sheet.cell(row = row, column = 3).value != None:
                        for i in range(2, numgroups+2):
                            numWells += int(sheet.cell(row = row, column = i).value)
                        break
                    else:
                        numWells = int(sheet.cell(row = row, column = 2).value)

            numreps = 0
            for group in range(0, len(groups)):
                numrepsCounter = 0
                for column in range(2,numWells+2):
                    if str(sheet.cell(row = groupRow, column = column).value) == str(groups[group]):
                        numrepsCounter += 1
                if numrepsCounter > numreps:
                    numreps = numrepsCounter

            # Count number of calculated parameters
            for row in range(1, sheet.max_row+1): # Loop through rows to find row containing first parameter
                if sheet.cell(row = row, column = 1).value == 'Number of Spikes':
                    firstParamRow = row
                    break
            for row in range(1, sheet.max_row+1): # Loop through rows to find row containing last parameter
                if sheet.cell(row = row, column = 1).value == 'Synchrony Index':
                    lastParamRow = row
                    break        

            numparams = lastParamRow - firstParamRow + 1

            # Determine number of days
            numdays = len(sheets)

            # Create NumPy array with dimensions: reps x groups x parameters x days
            array = np.ndarray(shape=(numreps,numgroups,numparams,numdays))

            ### Array indices ###
            # Reps: 0-x in order of Excel file
            # Groups: 0-x in order of Excel file
            # Parameters: 0-x = Parameters in order of Excel file from statistics compiler
            # Days: 0-x = Days in order of compiled Excel file sheets
            #####################

            for row in range(1, sheet.max_row+1): # Loop through rows to find row containing Well names
                if sheet.cell(row = row, column = 1).value == 'Well':
                    wellNameRow = row
                    break

            columns = OrderedDict()    # Create ORDERED dictionary for columns
            wells = OrderedDict()      # Create ORDERED dictionary for wells
            for i in groups:
                columns[i] = []   # Create empty column list for each group
                wells[i] = []     # Create empty well list for each group
                for column in range(2,numWells+2):
                    if sheet.cell(row = groupRow, column = column).value == i:    # Fill in column numbers and well names/numbers ('A1, A2', etc.) for each group
                        columns[i].append(column)
                        wells[i].append(str(sheet.cell(row = wellNameRow, column = column).value))

            ### Fill in array with all values ###
            for day in range(0, numdays):    # Loop through sheets (days)
                sheet = wb.worksheets[day]
                for row in range(1, sheet.max_row+1): # Loop through rows to find row containing first parameter
                    if sheet.cell(row = row, column = 1).value == 'Number of Spikes':
                        firstParamRow = row
                        break
                for row in range(1, sheet.max_row+1): # Loop through rows to find row containing last parameter
                    if sheet.cell(row = row, column = 1).value == 'Synchrony Index':
                        lastParamRow = row
                        break
                for param in range(0, numparams):   # Loop through rows (parameters)
                    row = (param + firstParamRow)
                    g = 0
                    for group in columns:
                        rep = 0
                        for c in columns[group]:
                            if sheet.cell(row = row, column = c).value == None:
                                array[rep][g][param][day] = 0
                            else:
                                array[rep][g][param][day] = sheet.cell(row = row, column = c).value
                            rep += 1
                        g += 1

            # Create empty Pandas dataframe (for later conversion from NumPy array) 
            data = pd.DataFrame()
            df_write = pd.ExcelWriter(platePath + '\\' + '2_AnalyzedAllParameters_' + plateFolder + '.xlsx')
            for p in range(0,numparams):
                paramname = str(sheet.cell(row = (p + firstParamRow), column = 1).value)
                xdays = []
                sheets_strings = []
                for s in sheets:
                    sheets_strings.append(str(s))
                for x in range(0, numdays):
                    xdays.append(sheets_strings[x][3:6])

                ### Slice array for desired parameter and days then convert to pandas dataframe ###
                array_sliced = array[:,:,p,:]
                array_meshgrid = np.column_stack(list(map(np.ravel, np.meshgrid(*map(np.arange, array_sliced.shape), indexing="ij"))) + [array_sliced.ravel()])

                datanew = pd.DataFrame(array_meshgrid, columns = ['Rep', 'Group', 'Day', paramname])   # Convert numpy array to pandas dataframe

                datanew['Rep'] += 1

                groupnames = {}
                for i in range(0, numgroups):
                    groupnames[i] = groups[i]
                datanew['Group'].replace(groupnames, inplace = True)

                daysdict = {}
                for i in range(0, numdays):
                    daysdict[i] = xdays[i]
                datanew['Day'].replace(daysdict, inplace = True)

                if p == 0:
                    data['Rep'] = datanew['Rep']
                    data['Group'] = datanew['Group']
                    data['Day'] = datanew['Day']
                data['Plate'] = plateFolder
                data[paramname] = datanew[paramname]
                data.to_excel(df_write, "All Parameters", index=False)
                #print('Analyzed Parameter ',p,'/',numparams)
            df_write.save()
            
            # Add separate compound and concentration columns
            wb = xl.load_workbook(platePath + '\\' + '2_AnalyzedAllParameters_' + plateFolder + '.xlsx')
            sheet = wb.worksheets[0]
            sheet.insert_cols(4, amount=3)
            sheet.cell(row = 1, column = 4).value = 'Compound'
            sheet.cell(row = 1, column = 5).value = 'Dose'
            sheet.cell(row = 1, column = 6).value = 'Units'
            for row in range(2, sheet.max_row+1):
                sheet.cell(row = row, column = 4).value = str(sheet.cell(row = row, column = 2).value).partition(' - ')[0]
                sheet.cell(row = row, column = 5).value = str(sheet.cell(row = row, column = 2).value).partition(' - ')[2].partition(' ')[0]
                sheet.cell(row = row, column = 6).value = str(sheet.cell(row = row, column = 2).value).partition(' - ')[2].partition(' ')[2]
            wb.save(platePath + '\\' + '2_AnalyzedAllParameters_' + plateFolder + '.xlsx')
    print('Analysis Excel files exported.')

In [ ]:
analyzeStatCompilerFiles(experimentPath)

In [ ]:
def mergePlateOutputFiles(experimentPath):
    # List compound subfolders
    compoundFolders = os.listdir(experimentPath)
    compoundFolders = [x for x in compoundFolders if '.datastreams' not in x]
    compoundFolders = [x for x in compoundFolders if '.xlsx' not in x]
    compoundPaths = [experimentPath + '\\' + x for x in compoundFolders]

    for path in tqdm(compoundPaths):
        plateFolders = os.listdir(path)
        plateFolders = [x for x in plateFolders if '.csv' not in x]
        plateFolders = [x for x in plateFolders if '.raw' not in x]
        plateFolders = [x for x in plateFolders if '.txt' not in x]
        plateFolders = [x for x in plateFolders if '.xlsx' not in x]
        plateFolders = [x for x in plateFolders if 'Compiled' not in x]

        compiledPath = path + "\\Compiled"
        if not os.path.exists(compiledPath):
            os.makedirs(compiledPath)

        # Check if plates are identical
        print(path)
        identicalPlates = input("Are plates identical [y/n]? ")
        if identicalPlates == 'y':
            numplates = int(input('How many identical plates are there [int]? '))
            if not isinstance(numplates, int):
                raise Warning("Invalid response. Answer must be an integer. Exiting...")
                sys.exit()
        elif identicalPlates == 'n':
            pass
        else:
            raise Warning("Invalid response. Answer must be 'y' or 'n'. Exiting...")
            sys.exit()

        files2_AnalyzedAllParameters = []
        for plateFolder in plateFolders:
            if 'platemap' in plateFolder:	# Ignore platemap file(s)
                continue
            else:
                platePath = path + '\\' + plateFolder

            # Create lists of .xlsx filenames to be merged
            files2_AnalyzedAllParameters.append(platePath + '\\' + '2_AnalyzedAllParameters_' + plateFolder + '.xlsx')

        # Open .xlsx file(s) and fix rep numbers for concatenation (e.g. Reps 1-6 on 2 plates become Reps 1-12 in compiled file)
        if identicalPlates == 'y':
            for i in range(0,len(files2_AnalyzedAllParameters)):
                wb = xl.load_workbook(files2_AnalyzedAllParameters[i])
                sheet = wb.worksheets[0]
                repCol = 0
                for column in range(1,sheet.max_column+1):
                    if sheet.cell(row = 1, column = column).value == 'Rep':
                        repCol = column
                maxRep = 0
                for row in range(2, sheet.max_row+1):
                    if sheet.cell(row = row, column = repCol).value > maxRep:
                        maxRep = sheet.cell(row = row, column = repCol).value
                for row in range(2, sheet.max_row+1):
                    sheet.cell(row = row, column = repCol).value += (maxRep * i)
                wb.save(compiledPath + '\\2_AnalyzedAllParameters_' + plateFolders[i] + '_RepsCorrected.xlsx')
                files2_AnalyzedAllParameters[i] = compiledPath + '\\2_AnalyzedAllParameters_' + plateFolders[i] + '_RepsCorrected.xlsx'

        # Read .xlsx files as pandas dataframes, then append to lists for concatenation
        list2_AnalyzedAllParameters = []
        for file in files2_AnalyzedAllParameters:
            list2_AnalyzedAllParameters.append(pd.read_excel(file))

        # Concatenate dataframes
        df2 = pd.concat(list2_AnalyzedAllParameters)

        # Export to .xlsx
        df2.to_excel(compiledPath + "\\2_AnalyzedAllParameters_Compiled.xlsx", index=False)
    print("Excel files merged.")

In [ ]:
mergePlateOutputFiles(experimentPath)

In [ ]:
def normZscore(experimentPath):
    # List compound subfolders
    compoundFolders = os.listdir(experimentPath)
    compoundFolders = [x for x in compoundFolders if '.datastreams' not in x]
    compoundFolders = [x for x in compoundFolders if '.xlsx' not in x]
    compoundPaths = [experimentPath + '\\' + x for x in compoundFolders]

    for path in tqdm(compoundPaths):
        # List plate subfolders
        plateFolders = os.listdir(path)
        plateFolders = [x for x in plateFolders if '.csv' not in x]
        plateFolders = [x for x in plateFolders if '.raw' not in x]
        plateFolders = [x for x in plateFolders if '.txt' not in x]
        plateFolders = [x for x in plateFolders if '.xlsx' not in x]
        
        # Loop through each plate folder
        for plateFolder in plateFolders:
            if 'platemap' in plateFolder:	# Ignore platemap file(s)
                continue
            else:
                platePath = path + '\\' + plateFolder	
                
            wb = xl.load_workbook(platePath + '\\' + '2_AnalyzedAllParameters_' + plateFolder + '.xlsx') 		# Load workbook
            sheets = wb.sheetnames							# Create list of all worksheets

            for page in range(0, len(sheets)):	 			# Loop through sheets (days)
                sheet = wb.worksheets[page]
                for col in range(8,sheet.max_column+1):
                    # print("Sheet: " + str(page + 1) + " | Column: " + str(col))
                    colvals = []
                    for row in range(2,sheet.max_row+1):
                        # print("Sheet: " + str(sheet) + " | " + "Column: " + str(col) + " | " + "Row: " + str(row))
                        if sheet.cell(row = row, column = col).value == None:
                            pass
                        elif type(sheet.cell(row = row, column = col).value) == str:
                            pass	
                        else:
                            colvals.append(sheet.cell(row = row, column = col).value)
                    col_mean = np.mean(colvals)
                    col_std = np.std(colvals)
                    for row in range(2,sheet.max_row+1):
                        if sheet.cell(row = row, column = col).value == None:
                            pass
                        elif type(sheet.cell(row = row, column = col).value) == str:
                            pass
                        else:
                            sheet.cell(row = row, column = col).value = ((sheet.cell(row = row, column = col).value - col_mean) / col_std)

            wb.save(platePath + '\\' + '3_Zscore_' + plateFolder + '.xlsx')
    print('Z-score Excel files exported.')

In [ ]:
normZscore(experimentPath)

In [ ]:
def NAS(experimentPath):
    # List compound subfolders
    compoundFolders = os.listdir(experimentPath)
    compoundFolders = [x for x in compoundFolders if '.datastreams' not in x]
    compoundFolders = [x for x in compoundFolders if '.xlsx' not in x]
    compoundFolders = [x for x in compoundFolders if '.pzfx' not in x]
    compoundFolders = [x for x in compoundFolders if '.csv' not in x]
    compoundPaths = [experimentPath + '\\' + x for x in compoundFolders]

    for path in tqdm(compoundPaths):
        # List plate subfolders
        plateFolders = os.listdir(path)
        plateFolders = [x for x in plateFolders if '.csv' not in x]
        plateFolders = [x for x in plateFolders if '.raw' not in x]
        plateFolders = [x for x in plateFolders if '.txt' not in x]
        plateFolders = [x for x in plateFolders if '.xlsx' not in x]

        # Loop through each plate folder
        for plateFolder in plateFolders:
            if 'platemap' in plateFolder:	# Ignore platemap file(s)
                continue
            else:
                platePath = path + '\\' + plateFolder

            wb = xl.load_workbook(platePath + '\\' + '3_Zscore_' + plateFolder + '.xlsx') 		# Load workbook
            sheets = wb.sheetnames   # Create list of all worksheets
            sheet = wb.worksheets[0]

            for col in range(1,sheet.max_column+1):
                if sheet.cell(row = 1, column = col).value == 'Mean Firing Rate (Hz)':
                    meanFiringRate = col
                    continue
                elif sheet.cell(row = 1, column = col).value == 'ISI Coefficient of Variation':
                    ISIcoeffVar = col
                    continue
                elif sheet.cell(row = 1, column = col).value == 'ISI Coefficient of Variation - Avg':
                    ISIcoeffVar = col
                    continue
                elif sheet.cell(row = 1, column = col).value == 'Number of Bursting Electrodes':
                    numBurstingElecs = col
                    continue
                elif sheet.cell(row = 1, column = col).value == 'Burst Duration - Avg (s)':
                    burstDuration = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Number of Spikes per Burst - Avg':
                    numSpikesPerBurst = col
                    continue
                elif sheet.cell(row = 1, column = col).value == 'Mean ISI within Burst - Avg':
                    meanISIwithinBurst = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Median ISI within Burst - Avg':
                    medianISIwithinBurst = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Inter-Burst Interval - Avg (s)':
                    IBI = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Burst Frequency - Avg (Hz)':
                    burstFreq = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Normalized Duration IQR - Avg':
                    normDurationIQR = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'IBI Coefficient of Variation - Avg':
                    IBIcoeffVar = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Burst Percentage - Avg':
                    burstPercentage = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Network Burst Frequency (Hz)':
                    networkBurstFreq = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Network Burst Duration - Avg (sec)':
                    networkBurstDuration = col
                    continue
                elif sheet.cell(row = 1, column = col).value == 'Number of Spikes per Network Burst - Avg':
                    numSpikesPerNetworkBurst = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Number of Elecs Participating in Burst - Avg':
                    numElecsParticipatingInBurst = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Number of Spikes per Network Burst per Channel - Avg':
                    numSpikesPerNetworkBurstPerChannel = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Network Burst Percentage':
                    networkBurstPercentage = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Network IBI Coefficient of Variation':
                    networkIBIcoeffVar = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Network Normalized Duration IQR':
                    networkNormDurationIQR = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Area Under Normalized Cross-Correlation':
                    areaUnderNormalizedCrossCorrelation = col
                    continue
                elif sheet.cell(row = 1, column = col).value == 'Area Under Cross-Correlation':
                    areaUnderCrossCorrelation = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Width at Half Height of Normalized Cross-Correlation':
                    halfWidthNormalizedCrossCorrelation = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Width at Half Height of Cross-Correlation':
                    halfWidthCrossCorrelation = col
                    continue
                elif sheet.cell(row = 1, column = col).value == 'Half Width at Half Height of Normalized Cross-Correlation':
                    halfWidthNormalizedCrossCorrelation = col
                    continue                
                elif sheet.cell(row = 1, column = col).value == 'Half Width at Half Height of Cross-Correlation':
                    halfWidthCrossCorrelation = col
                    continue
                elif sheet.cell(row = 1, column = col).value == 'Synchrony Index':
                    syncIndex = col
                    continue                

            for page in range(0, len(sheets)):   # Loop through sheets (days)
                sheet = wb.worksheets[page]
                maxCol = sheet.max_column
                sheet.cell(row = 1, column = maxCol+1).value = "NAS"
                for row in range(2,sheet.max_row+1):
                    index_score = \
                    (sheet.cell(row = row, column = meanFiringRate).value * 0.78723) + \
                    (sheet.cell(row = row, column = ISIcoeffVar).value * 0.87546) + \
                    (sheet.cell(row = row, column = numBurstingElecs).value * 0.92474) + \
                    (sheet.cell(row = row, column = burstDuration).value * 0.73062) + \
                    (sheet.cell(row = row, column = numSpikesPerBurst).value * 0.92763) + \
                    (sheet.cell(row = row, column = meanISIwithinBurst).value * 0.02926) + \
                    (sheet.cell(row = row, column = medianISIwithinBurst).value * -0.05205) + \
                    (sheet.cell(row = row, column = IBI).value * -0.27591) + \
                    (sheet.cell(row = row, column = burstFreq).value * 0.59294) + \
                    (sheet.cell(row = row, column = normDurationIQR).value * 0.70740) + \
                    (sheet.cell(row = row, column = IBIcoeffVar).value * 0.64845) + \
                    (sheet.cell(row = row, column = burstPercentage).value * 0.93895) + \
                    (sheet.cell(row = row, column = networkBurstFreq).value * 0.50654) + \
                    (sheet.cell(row = row, column = networkBurstDuration).value * 0.49079) + \
                    (sheet.cell(row = row, column = numSpikesPerNetworkBurst).value * 0.90668) + \
                    (sheet.cell(row = row, column = numElecsParticipatingInBurst).value * 0.86659) + \
                    (sheet.cell(row = row, column = numSpikesPerNetworkBurstPerChannel).value * 0.91682) + \
                    (sheet.cell(row = row, column = networkBurstPercentage).value * 0.93034) + \
                    (sheet.cell(row = row, column = networkIBIcoeffVar).value * 0.73943) + \
                    (sheet.cell(row = row, column = networkNormDurationIQR).value * 0.61389) + \
                    (sheet.cell(row = row, column = areaUnderNormalizedCrossCorrelation).value * 0.86675) + \
                    (sheet.cell(row = row, column = areaUnderCrossCorrelation).value * 0.60665) + \
                    (sheet.cell(row = row, column = halfWidthNormalizedCrossCorrelation).value * 0.17513) + \
                    (sheet.cell(row = row, column = halfWidthCrossCorrelation).value * 0.23308) + \
                    (sheet.cell(row = row, column = syncIndex).value * 0.91299)
                    sheet.cell(row = row, column = maxCol+1).value = index_score

            # Compile index score in GraphPad format
            # Create sheet with only Rep, Group, Day, Index Score columns
            wb.create_sheet('NAS only')
            isSheet = wb['NAS only']
            for i in range(1,sheet.max_row+1):
                for j in [1,2,3,4,5,6]:
                    isSheet.cell(row = i, column = j).value = sheet.cell(row = i, column = j).value
                isSheet.cell(row = i, column = 7).value = sheet.cell(row = i, column = maxCol+1).value
            
            # Remove outliers
            indexScoreData = isSheet.values
            indexScoreDataHeaders = next(indexScoreData)[0:]
            isDF = pd.DataFrame(indexScoreData, columns=indexScoreDataHeaders) # Convert to dataframe
            isDF['NAS (z)'] = stats.zscore(isDF['NAS']) # Z-score index score column
            isDFz = isDF['NAS (z)'].tolist()
            isSheet.cell(row = 1, column = 8).value = 'NAS (z)'
            for i in range(2, len(isDFz)+2):
                isSheet.cell(row = i, column = 8).value = isDFz[i-2]
            
            # Scale to 0-1
            isSheet.cell(row = 1, column = 9).value = 'NAS (scaled)'
            isVals = [isSheet.cell(row = x, column = 7).value for x in range(2,isSheet.max_row+1) if isSheet.cell(row = x, column = 8).value < 3 and isSheet.cell(row = x, column = 8).value > -3]
            minISval = min(isVals)
            maxISval = max(isVals)
            for i in range(2,isSheet.max_row+1):
                if isSheet.cell(row = i, column = 8).value > 3 or isSheet.cell(row = i, column = 8).value < -3:
                    isSheet.cell(row = i, column = 9).value = 'Outlier'
                else:
                    isSheet.cell(row = i, column = 9).value = (isSheet.cell(row = i, column = 7).value - minISval)/(maxISval - minISval)

            # Remove outliers and scale all other parameters
            scaledSheet = wb.copy_worksheet(sheet)
            scaledSheet.title = 'AllScaledParameters'
            for col in range(8,scaledSheet.max_column+1):
                if len(scaledSheet.cell(row = 1, column = col).value) > 30: # Shorten long parameter names to avoid Excel errors
                    if 'Avg' in scaledSheet.cell(row = 1, column = col).value:
                        if 'Channel' in scaledSheet.cell(row = 1, column = col).value:
                            scaledSheet.cell(row = 1, column = col).value = 'SpikesPerNetBurstPerChannel-Avg'
                        else:
                            scaledSheet.cell(row = 1, column = col).value = (str(scaledSheet.cell(row = 1, column = col).value)[:26] + '- Avg')
                    elif 'Std' in scaledSheet.cell(row = 1, column = col).value:
                        if 'Channel' in scaledSheet.cell(row = 1, column = col).value:
                            scaledSheet.cell(row = 1, column = col).value = 'SpikesPerNetBurstPerChannel-Std'
                        else:
                            scaledSheet.cell(row = 1, column = col).value = (str(scaledSheet.cell(row = 1, column = col).value)[:26] + '- Std')
                    elif 'IQR' in scaledSheet.cell(row = 1, column = col).value:
                        scaledSheet.cell(row = 1, column = col).value = (str(scaledSheet.cell(row = 1, column = col).value)[:26] + '- IQR')
                    elif 'Normalized' in scaledSheet.cell(row = 1, column = col).value:
                        scaledSheet.cell(row = 1, column = col).value = (str(scaledSheet.cell(row = 1, column = col).value)[:25] + '- Norm')
            scaledData = scaledSheet.values
            scaledDataHeaders = next(scaledData)[0:]
            scaledDF = pd.DataFrame(scaledData, columns=scaledDataHeaders) # Convert to dataframe
            for p in range(8,sheet.max_column+1): # Loop through parameters
                paramVals = [scaledSheet.cell(row = r, column = p).value for r in range(2,scaledSheet.max_row+1) if scaledSheet.cell(row = r, column = p).value < 3 and scaledSheet.cell(row = r, column = p).value > -3]
                minParamVal = min(paramVals)
                maxParamVal = max(paramVals)
                for r in range(2,scaledSheet.max_row+1):
                    if scaledSheet.cell(row = r, column = p).value > 3 or scaledSheet.cell(row = r, column = p).value < -3:
                        scaledSheet.cell(row = r, column = p).value = 'Outlier'
                    else:
                        scaledSheet.cell(row = r, column = p).value = (scaledSheet.cell(row = r, column = p).value - minParamVal)/(maxParamVal - minParamVal)
            
            # Replace NAS column on scaled sheet with scaled NAS
            for r in range(2,scaledSheet.max_row+1):
                scaledSheet.cell(row = r, column = maxCol+1).value = isSheet.cell(row = r, column = 9).value
            
            # Convert all parameters to GraphPad format
            for parameter in range(8,scaledSheet.max_column+1):
                # Convert to dataFrame
                scaledData = scaledSheet.values
                scaledDataHeaders = next(scaledData)[0:]
                dfScaled = pd.DataFrame(scaledData, columns=scaledDataHeaders)
                paramName = scaledSheet.cell(row = 1, column = parameter).value
                wb.create_sheet(paramName) # Create sheet for GraphPad format
                paramSheet = wb[paramName]
                dfScaled.sort_values(by=['Group', 'Day', 'Rep'], inplace=True) # Sort dataframe
                repList = []
                groupList = []
                dayList = []
                for index, row in dfScaled.iterrows(): # Loop through rows of sorted dataframe
                    if row[0] not in repList: # Create list of reps
                        repList.append(row[0])
                    if row[1] not in groupList: # Create list of groups
                        groupList.append(row[1])
                    if row[2] not in dayList: # Create list of days
                        dayList.append(row[2])

                # Fill in GraphPad format sheet
                paramSheet.cell(row = 1, column = 1).value = 'DIV'
                for i in range(2,len(dayList)+2):
                    paramSheet.cell(row = i, column = 1).value = dayList[i-2]
                groupCount, dayCount, repCount = 1, 1, 1
                for index, row in dfScaled.iterrows():
                    paramSheet.cell(row = 1, column = ((groupCount-1)*len(repList)) + repCount + 1).value = row[1]
                    if groupCount < len(groupList)+1:
                        if dayCount < len(dayList):
                            if repCount < len(repList):
                                paramSheet.cell(row = dayCount+1, column = ((groupCount-1)*len(repList)) + repCount + 1).value = row[parameter-1]
                                repCount += 1
                            elif repCount == len(repList):
                                paramSheet.cell(row = dayCount+1, column = ((groupCount-1)*len(repList)) + repCount + 1).value = row[parameter-1]
                                dayCount += 1 # Add to day counter
                                repCount = 1 # Reset rep counter
                        elif dayCount == len(dayList):
                            if repCount < len(repList):
                                paramSheet.cell(row = dayCount+1, column = ((groupCount-1)*len(repList)) + repCount + 1).value = row[parameter-1]
                                repCount += 1
                            elif repCount == len(repList):
                                paramSheet.cell(row = dayCount+1, column = ((groupCount-1)*len(repList)) + repCount + 1).value = row[parameter-1]
                                paramSheet.merge_cells(start_row=1, start_column=((groupCount-1)*len(repList) + 2), \
                                end_row=1, end_column=((groupCount-1)*len(repList) + 1 + len(repList))) # ...Merge group header cells before moving to next group
                                groupCount += 1 # Add to group counter
                                dayCount = 1 # Reset day counter
                                repCount = 1 # Reset rep counter
                    elif groupCount == len(groupList)+1:
                        break

            
            wb.save(platePath + '\\' + '4_NAS_' + plateFolder + '.xlsx')
    print('Excel file exported with NAS and all parameters scaled with outliers removed.')

In [ ]:
NAS(experimentPath)

In [ ]:
def allAUC(experimentPath):
    ### Calculates area under curve for all parameters over time for each group ###

    # List compound subfolders
    compoundFolders = os.listdir(experimentPath)
    compoundFolders = [x for x in compoundFolders if '.datastreams' not in x]
    compoundFolders = [x for x in compoundFolders if '.xlsx' not in x]
    compoundFolders = [x for x in compoundFolders if '.pzfx' not in x]
    compoundFolders = [x for x in compoundFolders if '.csv' not in x]
    compoundPaths = [experimentPath + '\\' + x for x in compoundFolders]

    for path in tqdm(compoundPaths):
        # List plate subfolders
        plateFolders = os.listdir(path)
        plateFolders = [x for x in plateFolders if '.csv' not in x]
        plateFolders = [x for x in plateFolders if '.raw' not in x]
        plateFolders = [x for x in plateFolders if '.txt' not in x]
        plateFolders = [x for x in plateFolders if '.xlsx' not in x]

        # Loop through each plate folder
        for plateFolder in plateFolders:
            if 'platemap' in plateFolder:	# Ignore platemap file(s)
                continue
            else:
                platePath = path + '\\' + plateFolder
            wb = xl.load_workbook(platePath + '\\' + '4_NAS_' + plateFolder + '.xlsx') 		# Load workbook

            # Determine number of reps
            sheet = wb.worksheets[0]  # First sheet with all parameters
            for col in range(1, sheet.max_column+1):
                if sheet.cell(row = 1, column = col).value == 'Rep':
                    repCol = col
                    break
            numReps = 0
            for row in range(2, sheet.max_row+1):
                if int(sheet.cell(row = row, column = repCol).value) > numReps:
                    numReps = sheet.cell(row = row, column = repCol).value

            # Calculate AUC values for all parameters
            for sheet in wb.worksheets[3:]:
                maxRow = sheet.max_row
                sheet.cell(row = maxRow+1, column = 1).value = 'Group'
                sheet.cell(row = maxRow+2, column = 1).value = 'AUC'
                for col in range(2, sheet.max_column+1):
                    sheet.cell(row = 6, column = col).value = sheet.cell(row = 1, column = col).value
                xVals = []
                for row in range(2, maxRow+1):
                    xVals.append(int(sheet.cell(row = row, column = 1).value))
                for col in range(2, sheet.max_column+1):
                    yVals = []
                    for row in range(2, maxRow+1):
                        yVals.append(sheet.cell(row = row, column = col).value)
                    xValsCorrected = []
                    yValsCorrected = []
                    for i in range(0,len(yVals)):
                        if yVals[i] != 'Outlier':
                            xValsCorrected.append(xVals[i])
                            yValsCorrected.append(yVals[i])
                    sheet.cell(row = 7, column = col).value = np.trapz(yValsCorrected, x = xValsCorrected)

                # Normalize to control
                sheet.cell(row = maxRow+3, column = 1).value = 'AUC (% of Control)'
                if 'Compiled' in platePath:
                    compoundColIndices = [2, 26, 50, 74, 98, 122]
                    aucValNum = 24
                    for compound in compoundColIndices:
                        for aucVal in range(0,aucValNum):
                            sheet.cell(row = maxRow+3, column = compound + aucVal).value = (sheet.cell(row = maxRow+2, column = compound + aucVal).value / ((sheet.cell(row = maxRow+2, column = compound).value + sheet.cell(row = maxRow+2, column = compound + 1).value + sheet.cell(row = maxRow+2, column = compound + 2).value) / 3)) * 100
                else:
                    compoundColIndices = [2, 10, 18, 26, 34, 42]
                    aucValNum = 8
                    for compound in compoundColIndices:
                        for aucVal in range(0,aucValNum):
                            sheet.cell(row = maxRow+3, column = compound + aucVal).value = (sheet.cell(row = maxRow+2, column = compound + aucVal).value / sheet.cell(row = maxRow+2, column = compound).value) * 100

                # GraphPad format
                sheet.cell(row = maxRow+5, column = 1).value = 'GraphPad Format'
                sheet.cell(row = maxRow+5+1, column = 1).value = 'Dose'
                sheet.cell(row = maxRow+5+2, column = 1).value = 0
                sheet.cell(row = maxRow+5+3, column = 1).value = 0.03
                sheet.cell(row = maxRow+5+4, column = 1).value = 0.1
                sheet.cell(row = maxRow+5+5, column = 1).value = 0.3
                sheet.cell(row = maxRow+5+6, column = 1).value = 1
                sheet.cell(row = maxRow+5+7, column = 1).value = 3
                sheet.cell(row = maxRow+5+8, column = 1).value = 10
                sheet.cell(row = maxRow+5+9, column = 1).value = 20
                sheet.cell(row = maxRow+5+1, column = 2).value = sheet.cell(row = 1, column = 2).value.partition(' - ')[0]
                lastCompoundColWritten = 2
                if 'Compiled' in platePath:
                    for col in range(3, sheet.max_column+1):
                        if sheet.cell(row = maxRow+1, column = col).value == None:
                            continue
                        elif sheet.cell(row = maxRow+1, column = col).value.partition(' - ')[0] != sheet.cell(row = maxRow+5+1, column = lastCompoundColWritten).value:
                            groupVal = sheet.cell(row = maxRow+1, column = col).value
                            sheet.cell(row = maxRow+5+1, column = lastCompoundColWritten + 3).value = groupVal.partition(' - ')[0]
                            lastCompoundColWritten = lastCompoundColWritten + 3
                else:
                    for col in range(3, sheet.max_column+1):
                        if sheet.cell(row = maxRow+1, column = col).value == None:
                            continue
                        elif sheet.cell(row = maxRow+1, column = col).value.partition(' - ')[0] != sheet.cell(row = maxRow+5+1, column = lastCompoundColWritten).value:
                            groupVal = sheet.cell(row = maxRow+1, column = col).value
                            sheet.cell(row = maxRow+5+1, column = lastCompoundColWritten + 1).value = groupVal.partition(' - ')[0]
                            lastCompoundColWritten = lastCompoundColWritten + 1   
                if 'Compiled' in platePath:
                    aucCompoundColIndices = [2, 26, 50, 74, 98, 122]
                    gpCompoundColIndices = [2, 5, 8, 11, 14, 17]
                    for compound in range(0,6):
                        for dose in range(0,8):
                            for rep in range(0,3):
                                sheet.cell(row = maxRow+5+dose + 2, column = gpCompoundColIndices[compound] + rep).value = sheet.cell(row = maxRow+3, column = aucCompoundColIndices[compound] + (dose*3) + rep).value
                else:
                    aucCompoundColIndices = [2, 10, 18, 26, 34, 42]
                    gpCompoundColIndices = [2, 3, 4, 5, 6, 7]
                    for compound in range(0,6):
                        for dose in range(0,8):
                            sheet.cell(row = maxRow+5+dose + 2, column = gpCompoundColIndices[compound]).value = sheet.cell(row = maxRow+3, column = aucCompoundColIndices[compound] + dose).value
            wb.save(platePath + '\\' + '5_NAS_AUC_' + plateFolder + '.xlsx')
    print('AUC Excel files exported.')

In [ ]:
allAUC(experimentPath)

In [ ]:
def mergeAllAUC(experimentPath):
    ### Merges AUC files for 'Compiled' folders to create file with all compounds
    
    # List compound subfolders
    compoundFolders = os.listdir(experimentPath)
    compoundFolders = [x for x in compoundFolders if '.datastreams' not in x]
    compoundFolders = [x for x in compoundFolders if '.xlsx' not in x]
    compoundFolders = [x for x in compoundFolders if '.pzfx' not in x]
    compoundFolders = [x for x in compoundFolders if '.csv' not in x]
    compoundPaths = [experimentPath + '\\' + x for x in compoundFolders]
    
    compiledPaths = []
    for path in compoundPaths: # List all compiled AUC files
        compiledPaths.append(path + '\\Compiled\\5_NAS_AUC_Compiled.xlsx')
    wb1 = xl.load_workbook(compiledPaths[0]) # Load first workbook to start
    filenum = 0
    for filepath in tqdm(compiledPaths[1:]):
        wb2 = xl.load_workbook(filepath)
        for sheet in range(3,len(wb1.worksheets)):
            if filenum == 0:
                wb1.worksheets[sheet].delete_rows(1,10) # Delete all but GraphPad format rows from wb1
            wb2.worksheets[sheet].delete_rows(1,10) # Delete all but GraphPad format rows from wb2
            
            # Append values from wb2 to wb1
            for r in range(1, 10):
                for c in range(2, 20):
                    wb1.worksheets[sheet].cell(row = r, column = c + 18 + filenum).value = wb2.worksheets[sheet].cell(row = r, column = c).value
        filenum += 18
    wb1.save(experimentPath + '/6_AllCompounds_AllAUC.xlsx')
    print('Compiled file exported.')

In [ ]:
mergeAllAUC(destinationPath)

**Experiment analysis complete.**